In [1]:
from mesh_to_sdf import mesh_to_sdf

import trimesh
import pyrender
import numpy as np

tfs = np.eye(4)
# First is small translation to get the anchor to front left corner
# Then translate anchor to desired location
tfs[:3,3] = np.array([5,10,-2.5]) + np.array([0, 0, 10])

sm = trimesh.creation.box(np.array([10.0, 20.0, 5.0]), transform=tfs)
sm.visual.vertex_colors = [0.0, 1.0, 0.0, 0.1]

bounds = [15, 20, 15]
increment = 0.5

x = np.arange(0, bounds[0], increment)
y = np.arange(0, bounds[1], increment)
z = np.arange(0, bounds[2], increment)

sdf_points = []

for xi in x:
    for yi in y:
        for zi in z:
            sdf_points.append([xi, yi, zi])

sdf_points = np.array(sdf_points)


In [2]:
sdf = mesh_to_sdf(sm, sdf_points)
print(sdf.shape)

(36000,)


In [3]:
colors = np.zeros(sdf_points.shape)
colors[sdf < 0, 2] = 1
colors[sdf > 0, 0] = 1
cloud = pyrender.Mesh.from_points(sdf_points, colors=colors)
scene = pyrender.Scene()
scene.add(cloud)
viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2, show_world_axis=True)

In [4]:

sdf_new = np.reshape(sdf, (x.shape[0],y.shape[0],z.shape[0]))
print(sdf_new.shape)
np.save("box_sdf.npy", sdf_new)


(30, 40, 30)


In [17]:
sdf_grad = np.array(np.gradient(sdf_new, x, y, z)) # This gives the volume normals
sdf_grad = np.moveaxis(sdf_grad, 0, -1)
print(sdf_grad.shape)
np.save("box_sdf_grad.npy", sdf_grad)


(30, 40, 30, 3)


In [18]:
def find_nearest_ind(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def compute_ind(low_bound, up_bound, inc, v):
    if v <= low_bound:
        return 0
    elif v >= up_bound:
        return -1
    else:
        shifted_v = v - low_bound
        return np.floor(shifted_v / inc).astype(int)

def get_inds(x, y, z, p):
    inds = [0, 0, 0]
    # inds[0] = find_nearest_ind(x,p[0])
    # inds[1] = find_nearest_ind(y,p[1])
    # inds[2] = find_nearest_ind(z,p[2])
    inds[0] = compute_ind(0, bounds[0], increment, p[0])
    inds[1] = compute_ind(0, bounds[1], increment, p[1])
    inds[2] = compute_ind(0, bounds[2], increment, p[2])
    inds = np.array(inds)
    return inds

def get_val(x, y, z, sdf, p):
    inds = get_inds(x, y, z, p)
    val = sdf[inds[0], inds[1], inds[2]]
    return val

def get_grads(x, y, z, sdf_grad, p):
    ind = get_inds(x, y, z, p)
    grad = [0, 0, 0]
    grad[0] = sdf_grad[ind[0], ind[1], ind[2], 0]
    grad[1] = sdf_grad[ind[0], ind[1], ind[2], 1]
    grad[2] = sdf_grad[ind[0], ind[1], ind[2], 2]
    grad = np.array(grad)
    return grad


In [19]:
# Exterior
p = [12, 10, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))

# Interior
p = [2, 5, 9]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))

# Edge
p = [10, 1, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))


1.9686514
[ 0.9995166  -0.00371587 -0.00238073]
-0.9996663
[-5.041361e-04  3.746748e-04  9.994429e-01]
-0.020375414
[ 9.7817600e-01 -5.1860511e-04 -1.7936803e-02]


In [20]:
def collision_correction(p, x, y, z, sdf, sdf_grad):
    correction = np.array([0.0, 0.0, 0.0])
    sdf_val = get_val(x, y, z, sdf, p)
    normal = get_grads(x, y, z, sdf_grad, p)
    # Negative SDF value means inside the solid
    if sdf_val <= 0:
        correction = -1 * sdf_val * normal
    return correction

In [21]:
# Exterior
p = [12, 10, 7.5]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))

# Interior
p = [2, 5, 9]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))

# Edge
p = [10, 1, 7.5]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))


[0. 0. 0.]
[-5.0396787e-04  3.7454977e-04  9.9910933e-01]
[ 1.9930741e-02 -1.0566794e-05 -3.6546978e-04]
